In [0]:

%run "/Workspace/CODES/ADB_PROJ ONE/AZURE PROJ_1 ADB/Utilities/Utilities"

In [0]:
from pyspark.sql.functions import current_timestamp
checkpoint_loc="/mnt/cleansed_datalake/PLANE"

In [0]:
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "csv")
      .option("cloudFiles.inferColumnTypes", "true")
      .option("cloudFiles.schemaLocation",checkpoint_loc )
      .load("dbfs:/mnt/raw_datalake/PLANE/")
      .withColumn("Date_Part",current_timestamp())
      )


In [0]:

df_base = df.selectExpr(
    "tailnum as tailid",
    "type",
    "manufacturer",
    "to_date(issue_date) as issue_date",
    "model",
    "status",
    "aircraft_type",
    "engine_type",
    "cast(year as int) as year",
    "to_date(Date_Part,'yyyy-MM-dd') as Date_Part"
)
df_base.writeStream.trigger(availableNow=True).format("delta").option(
    "checkpointLocation", checkpoint_loc
).start("/mnt/cleansed_datalake/PLANE")

In [0]:
f_delta_cleansed_load("plane", "/mnt/cleansed_datalake/PLANE","cleansed_azurep1")